In [1]:
import torch
from torch import nn, optim
from torchvision.transforms import functional as TF
import os
from PIL import Image
import numpy as np
import kornia


## The Vector Heat Method

### One implementation with Pytorch

In [ ]:

# This code is given by the following repository: https://github.com/jakericedesigns/Pytorch-Heat-Method/. The implementation is pleasantly straigthforward.

def solve_poisson(img, iters=1000):
    LU = torch.tensor([[[[0, 1., 0],
                         [1, 0,  1],
                         [0, 1,  0]]]], dtype=img.dtype).to(img.device)

    BC = nn.ReplicationPad2d(1)
    solution = img
    for i in range(iters):
            solution = (img - nn.functional.conv2d(BC(solution), LU)) / -4.0
    return solution

def screened_poisson(img, timestep=.1, mass=.01, iters=1000):
    LU = torch.tensor([[[[0, -1., 0],
                         [-1, 0, -1],
                         [0, -1,  0]]]], dtype=img.dtype).to(img.device)

    BC = nn.ReplicationPad2d(1)  
    solution = img
    for i in range(iters):
            solution = (img - nn.functional.conv2d(BC(solution), LU * timestep)) / (mass + 4.0 * timestep)   
    return solution    

def finite_diff_grad(img):
    kernel_x = torch.tensor([[[[0, 0., 0],
                               [1, 0, -1],
                               [0, 0,  0]]]], dtype=img.dtype).to(img.device)

    kernel_y= torch.tensor([[[[0, 1,  0],
                              [0, 0,  0],
                              [0, -1, 0]]]], dtype=img.dtype).to(img.device)
    div_x = nn.functional.conv2d(img, kernel_x)
    div_y = nn.functional.conv2d(img, kernel_y)
    return torch.cat((div_x, div_y), 1) / 2.0

def finite_diff_div(grad):
    kernel_x = torch.tensor([[[[0, 0., 0],
                               [1, 0, -1],
                               [0, 0,  0]]]], dtype=grad.dtype).to(grad.device)
    kernel_y= torch.tensor([[[[0, 1,  0],
                              [0, 0,  0],
                              [0, -1, 0]]]], dtype=grad.dtype).to(grad.device)

    div_x = nn.functional.conv2d(grad[:,0:1,...], kernel_x)
    div_y = nn.functional.conv2d(grad[:,1:,...], kernel_y)
    return (div_x + div_y) / 4.0

# Now we can define the heat method function:

def heat_method(image, timestep=1.0, mass=.01, iters_diffusion=500, iters_poisson=1000):
    heat = screened_poisson(image, timestep, mass, iters_diffusion) #diffusion step
    grad = finite_diff_grad(heat) * -1 #inverted gradient
    grad = grad / grad.norm(dim=1) #normalize gradient
    div = finite_diff_div(grad)
    distance = solve_poisson(div, iters_poisson)
    return distance

def fitrange(x):
    c_max = torch.max(x)
    c_min = torch.min(x)
    return (x - c_min) / (c_max - c_min)



Below are some snippets for the code to work, from the same source:

In [ ]:
if __name__ == "__main__":
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    
    init_image = "H:\\Mon Drive\\Ai M2\\S2\\Minimal Paths\\Projet\\raie.jpg" ##Path to the image


    pil_image = Image.open(init_image).convert('RGB') #open image
    image = TF.to_tensor(pil_image).to(device).unsqueeze(0) #convert to tensor and add batch dimension

    #extract edges from our input images (this is a nice add by Jack Rice but not necessary for the heat method to work)
    edges = kornia.filters.canny(image)[1]
    
    #perform heat method to get depth map
    depth_map = heat_method(edges, timestep=0.1, mass=.01, iters_diffusion=500, iters_poisson=1000)


In [ ]:
# save the output images (change the path to save the images in the desired location)
#TF.to_pil_image(edges[0].cpu()).save('H:\\Mon Drive\\Ai M2\\S2\\Minimal Paths\\Projet\\edgesraie_out.jpg') 
#TF.to_pil_image(fitrange(depth_map)[0].cpu()).save('H:\\Mon Drive\\Ai M2\\S2\\Minimal Paths\\Projet\\depthraie_out.jpg')